In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, get_scheduler
from trl import SFTTrainer, SFTConfig
import torch

from dotenv import load_dotenv
load_dotenv()

import json
import pickle
import random

from utils_simulate import run_games

import os
from huggingface_hub.hf_api import HfFolder
HfFolder.save_token(os.environ['HUGGINGFACE_TOKEN'])

import datasets

import pandas as pd

# from transformers.utils import logging
# logging.set_verbosity_info()

import logging
from transformers.utils import logging as transformers_logging

class CustomFilter(logging.Filter):
    def filter(self, record):
        return 'right-padding was detected' not in record.getMessage()

logger = logging.getLogger('transformers')

for handler in logger.handlers:
    handler.addFilter(CustomFilter())

In [ ]:
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
# model_id = "astronomer/Llama-3-8B-Special-Tokens-Adjusted"

# tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side='left')
tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side='right')
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
model.gradient_checkpointing_enable()
id_eot = tokenizer.convert_tokens_to_ids(["<|eot_id|>"])[0]
pad_token_id = tokenizer.convert_tokens_to_ids(["<|end_of_text|>"])[0]

tokenizer.pad_token_id = pad_token_id
model.config.pad_token_id = pad_token_id

# tokenizer.pad_token_id = id_eot
# model.config.pad_token_id = id_eot

# tokenizer.add_special_tokens({"pad_token": "<|padding_token|>"})
# model.config.pad_token_id = tokenizer.pad_token_id

In [ ]:
1

In [ ]:
tokenizer.pad_token, tokenizer.pad_token_id

In [ ]:
id_eot, pad_token_id

In [ ]:
d_basic = pickle.load(open("data_collection/!final_dataset_11194_basic.pkl", "rb"))
d_hints = pickle.load(open("data_collection/!final_dataset_6016_with_hints_after_n_step.pkl", "rb"))
d_snap = pickle.load(open("data_collection/!final_dataset_17476_from_snapshots_fixed.pkl", "rb"))

In [ ]:
def split_dataset(dataset, split_ratio=0.05):
    random.shuffle(dataset)
    split_index = int(len(dataset) * split_ratio)
    eval_set = dataset[:split_index]
    train_set = dataset[split_index:]
    return train_set, eval_set

d_basic, d_basic_eval = split_dataset(d_basic)
d_hints, d_hints_eval = split_dataset(d_hints)
d_snap, d_snap_eval = split_dataset(d_snap)

d_eval = d_basic_eval + d_hints_eval + d_snap_eval

In [ ]:
# # # TODO TEMP!
# d_basic = d_basic[:1000]
# d_hints = d_hints[:30]
# d_snap = d_snap[:30]
# d_eval = d_eval[:30]

In [ ]:
# def prepare_data(data):
#     prompts = [item['prompt'] for item in data]
#     responses = [item['response'] for item in data]
#     return datasets.Dataset.from_dict({"prompt": prompts, "response": responses})


from utils import SYSTEM_PROMPT_ASKER, SYSTEM_PROMPT_ANSWERER
# def prepare_inputs(chats: list[dict]):
#     tokenizer.padding_side = "left"
#     r_inputs = tokenizer.apply_chat_template(
#         [
#             [
#                 {"role": "system", "content": SYSTEM_PROMPT_ASKER if "only YES or NO" not in o['prompt'] else SYSTEM_PROMPT_ANSWERER},
#                 {"role": "user", "content": o['prompt']},
#             ] for o in chats
#         ],
#         tokenize=True,
#         padding="longest",
#         return_dict=True,
#     )
#     tokenizer.padding_side = "right"
#     r_labels = tokenizer.apply_chat_template(
#         [
#             [
#                 {"role": "assistant", "content": o['response']}
#             ] for o in chats
#         ],
#         tokenize=True,
#         padding="longest",
#         # return_dict=True,
#     )
#     return datasets.Dataset.from_dict(
#         {
#             "input_ids": r_inputs['input_ids'],
#             "attention_mask": r_inputs['attention_mask'],
#             "labels": r_labels,
#         }
#     ).shuffle(seed=42)


def prepare_inputs(chats: list[dict]):
    r_inputs = tokenizer.apply_chat_template(
        [
            [
                {"role": "system", "content": SYSTEM_PROMPT_ASKER if "only YES or NO" not in o['prompt'] else SYSTEM_PROMPT_ANSWERER},
                {"role": "user", "content": o['prompt']},                
                {"role": "assistant", "content": o['response']}
            ] for o in chats
        ],
        tokenize=False,
        padding=False,
        return_dict=False,
    )
    return datasets.Dataset.from_dict(
        {"texts": [r[:-len("<|start_header_id|>assistant<|end_header_id|>\n\n")] for r in r_inputs]}
    ).shuffle(seed=42)


# def tokenize_function(examples):
    # return tokenizer(examples['prompt'], text_target=examples['response'], truncation=True, padding="max_length", max_length=2500)

In [ ]:
ds_basic = prepare_inputs(d_basic)
print("ds_basic", len(ds_basic))

ds_hints = prepare_inputs(d_hints)
print("d_hints", len(d_hints))

ds_snap = prepare_inputs(d_snap)
print("ds_snap", len(ds_snap))

ds_eval = prepare_inputs(d_eval)
print("ds_eval", len(ds_eval))

In [ ]:
for name, param in model.named_parameters():
    if "model.layers." not in name and not "head" in name and not "embed_tokens" in name:
        param.requires_grad = False
        continue
    if "mlp" in name:
        param.requires_grad = False
        continue
    if "norm" in name:
        param.requires_grad = False
        continue
    if "head" in name or "embed_tokens" in name:
        continue
    layer_num = int(name.split(".")[2])
    if layer_num <= 21:
        param.requires_grad = False
        continue

# for name, param in model.named_parameters():
#     if "model.layers." not in name and not "head" in name:
#         param.requires_grad = False
#         continue
#     if "mlp" in name:
#         param.requires_grad = False
#         continue
#     if "norm" in name:
#         param.requires_grad = False
#         continue
#     if "head" in name:
#         continue
#     layer_num = int(name.split(".")[2])
#     if layer_num <= 20:
#         param.requires_grad = False
#         continue

In [ ]:
for name, param in model.named_parameters():
    print(name, param.requires_grad)

In [ ]:
def count_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

num_trainable_params = count_trainable_parameters(model)
print(f"Number of trainable parameters: {num_trainable_params}")

In [ ]:
import logging
logging.basicConfig(filename='training_log.txt', level=logging.INFO)

In [ ]:
# eval_set = json.load(open("data/eval_set.json", "r"))[:10] # TODO!!!!
eval_set = json.load(open("data/eval_set.json", "r"))
train_set = json.load(open("data/train_set.json", "r"))
def custom_evaluation(model, epoch, short=True):
    if short:
        keywords = [e['keyword'] for e in eval_set if e['difficulty'] == "Easy"]
    else:
        keywords = [e['keyword'] for e in eval_set]

    games = run_games(
        keywords,
        tokenizer,
        model,
        id_eot,
        batch_size=15,
    )
    if not short:
        pickle.dump(games, open(f"full_eval_{epoch}.pkl", "wb"))
    return sum([g.win for g in games]) / len(games)

def custom_evaluation_on_train(model, n_examples=100):
    import random
    random.seed(42)
    random.shuffle(train_set)
    
    keywords = [e['keyword'] for e in train_set[:n_examples]]
    games = run_games(
        keywords,
        tokenizer,
        model,
        id_eot,
        batch_size=15,
    )
    return sum([g.win for g in games]) / len(games)

In [ ]:
from trl.trainer import DataCollatorForCompletionOnlyLM

data_collator = DataCollatorForCompletionOnlyLM(response_template="<|start_header_id|>assistant<|end_header_id|>", tokenizer=tokenizer)

In [ ]:
model = model.train()

In [ ]:
save_epochs = set([0])
cumulative_epoch = 0
for ds, num_epochs in (
    # (ds_basic, 1),
    # (ds_hints, 1),
    (ds_snap, 1),
):
    for epoch in range(num_epochs):
        training_args = SFTConfig(
            output_dir="./results",
            num_train_epochs=1,
            per_device_train_batch_size=25,
            per_device_eval_batch_size=25,
            save_strategy="no",
            learning_rate=1e-05,
            weight_decay=0.2,
            bf16=True,
            lr_scheduler_type="linear",
            warmup_ratio=0.1,
            eval_strategy="steps",
            eval_steps=150,
            logging_steps=150,
            neftune_noise_alpha=None,
            max_seq_length=5000,
            num_of_sequences=5000,
            dataset_text_field="texts",
        )

        trainer = SFTTrainer(
            model=model,
            args=training_args,
            train_dataset=ds,
            eval_dataset=ds_eval,
            tokenizer=tokenizer,
            data_collator=data_collator,
        )

        train_result = trainer.train()
        train_loss = train_result.training_loss
        logging.info(f"Epoch {cumulative_epoch} - Train Loss: {train_loss}")
        print(f"Epoch {cumulative_epoch} - Train Loss: {train_loss}")
        
        eval_result = trainer.evaluate()
        eval_loss = eval_result['eval_loss']
        
        logging.info(f"Epoch {cumulative_epoch} - Eval Loss: {eval_loss}")
        print(f"Epoch {cumulative_epoch} - Eval Loss: {eval_loss}")
        
        eval_metrics = custom_evaluation(model, cumulative_epoch)
        logging.info(f"Epoch {cumulative_epoch} - Custom Eval: {eval_metrics}")
        print(f"Epoch {cumulative_epoch} - Custom Eval: {eval_metrics}")
        
        if cumulative_epoch in save_epochs:
            # TODO!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            # output_dir = f"./results/llama_{cumulative_epoch}"
            # os.makedirs(output_dir, exist_ok=True)
            # model.save_pretrained(output_dir)
            # tokenizer.save_pretrained(output_dir)
            eval_metrics = custom_evaluation(model, cumulative_epoch, short=False)
            logging.info(f"Epoch {cumulative_epoch} - Custom Eval (full): {eval_metrics}")
            print(f"Epoch {cumulative_epoch} - Custom Eval (full): {eval_metrics}")
            eval_metrics_on_train = custom_evaluation_on_train(model)
            logging.info(f"Epoch {cumulative_epoch} - Custom Eval (full train): {eval_metrics_on_train}")
            print(f"Epoch {cumulative_epoch} - Custom Eval (full train): {eval_metrics_on_train}")
        
        print(f"Finished epoch {cumulative_epoch + 1}")
        cumulative_epoch += 1  # Increment the overall epoch counter

In [ ]:
cumulative_epoch = 100
eval_metrics = custom_evaluation(model, cumulative_epoch)
logging.info(f"Epoch {cumulative_epoch} - Custom Eval: {eval_metrics}")
print(f"Epoch {cumulative_epoch} - Custom Eval: {eval_metrics}")
eval_metrics = custom_evaluation(model, cumulative_epoch, short=False)
logging.info(f"Epoch {cumulative_epoch} - Custom Eval (full): {eval_metrics}")
print(f"Epoch {cumulative_epoch} - Custom Eval (full): {eval_metrics}")
eval_metrics_on_train = custom_evaluation_on_train(model)
logging.info(f"Epoch {cumulative_epoch} - Custom Eval (full train): {eval_metrics_on_train}")
print(f"Epoch {cumulative_epoch} - Custom Eval (full train): {eval_metrics_on_train}")

In [ ]:
# eval_metrics = custom_evaluation(model, cumulative_epoch)

In [ ]:
# eval_metrics

In [ ]:
# output_dir = f"./results/llama_{cumulative_epoch}"
# os.makedirs(output_dir, exist_ok=True)
# model.save_pretrained(output_dir)
# tokenizer.save_pretrained(output_dir)

In [ ]:
# eval_metrics = custom_evaluation(model, 111, short=False)

In [ ]:
save_epochs = set([0])
cumulative_epoch = 0
for ds, num_epochs in (
    # (ds_basic, 1),
    (ds_hints, 1),
    # (ds_snap, 1),
):
    for epoch in range(num_epochs):
        training_args = SFTConfig(
            output_dir="./results",
            num_train_epochs=1,
            per_device_train_batch_size=25,
            per_device_eval_batch_size=25,
            save_strategy="no",
            learning_rate=1e-05,
            weight_decay=0.25,
            bf16=True,
            lr_scheduler_type="linear",
            warmup_ratio=0.1,
            eval_strategy="steps",
            eval_steps=150,
            logging_steps=150,
            neftune_noise_alpha=None,
            max_seq_length=5000,
            num_of_sequences=5000,
            dataset_text_field="texts",
        )

        trainer = SFTTrainer(
            model=model,
            args=training_args,
            train_dataset=ds,
            eval_dataset=ds_eval,
            tokenizer=tokenizer,
            # data_collator=data_collator,
        )

        train_result = trainer.train()
        train_loss = train_result.training_loss
        logging.info(f"Epoch {cumulative_epoch} - Train Loss: {train_loss}")
        print(f"Epoch {cumulative_epoch} - Train Loss: {train_loss}")
        
        eval_result = trainer.evaluate()
        eval_loss = eval_result['eval_loss']
        
        logging.info(f"Epoch {cumulative_epoch} - Eval Loss: {eval_loss}")
        print(f"Epoch {cumulative_epoch} - Eval Loss: {eval_loss}")
        
        eval_metrics = custom_evaluation(model, cumulative_epoch)
        logging.info(f"Epoch {cumulative_epoch} - Custom Eval: {eval_metrics}")
        print(f"Epoch {cumulative_epoch} - Custom Eval: {eval_metrics}")
        
        if cumulative_epoch in save_epochs:
            # TODO !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            # output_dir = f"./results/llama_no_mask_{cumulative_epoch}"
            # os.makedirs(output_dir, exist_ok=True)
            # model.save_pretrained(output_dir)
            # tokenizer.save_pretrained(output_dir)
            eval_metrics = custom_evaluation(model, cumulative_epoch, short=False)
            logging.info(f"Epoch {cumulative_epoch} - Custom Eval (full): {eval_metrics}")
            print(f"Epoch {cumulative_epoch} - Custom Eval (full): {eval_metrics}")
            eval_metrics_on_train = custom_evaluation_on_train(model)
            logging.info(f"Epoch {cumulative_epoch} - Custom Eval (full train): {eval_metrics_on_train}")
            print(f"Epoch {cumulative_epoch} - Custom Eval (full train): {eval_metrics_on_train}")
       
        print(f"Finished epoch {cumulative_epoch + 1}")
        cumulative_epoch += 1  # Increment the overall epoch counter

In [ ]:
# eval_metrics = custom_evaluation(model, cumulative_epoch)
# logging.info(f"Epoch {cumulative_epoch} - Custom Eval: {eval_metrics}")
# print(f"Epoch {cumulative_epoch} - Custom Eval: {eval_metrics}")
# eval_metrics = custom_evaluation(model, cumulative_epoch, short=False)
# logging.info(f"Epoch {cumulative_epoch} - Custom Eval (full): {eval_metrics}")
# print(f"Epoch {cumulative_epoch} - Custom Eval (full): {eval_metrics}")
# eval_metrics_on_train = custom_evaluation_on_train(model)
# logging.info(f"Epoch {cumulative_epoch} - Custom Eval (full train): {eval_metrics_on_train}")
# print(f"Epoch {cumulative_epoch} - Custom Eval (full train): {eval_metrics_on_train}")

In [ ]:
output_dir = f"./results/llama_no_mask_{cumulative_epoch}"
os.makedirs(output_dir, exist_ok=True)
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
eval_metrics = custom_evaluation(model, cumulative_epoch, short=False)
logging.info(f"Epoch {cumulative_epoch} - Custom Eval (full): {eval_metrics}")
print(f"Epoch {cumulative_epoch} - Custom Eval (full): {eval_metrics}")